In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.team_predicter import TeamPredicter, SolverMode
from modules.utils import getDataFilesSorted

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./data/results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./data/results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 0
TOTAL_BUDGET = 1000

In [3]:
current_team_names = {"Mark Flekken",
                      "Matz Sels",
                      "Dan Burn",
                      "Rayan Aït-Nouri",
                      "Daniel Muñoz",
                      "Ashley Young",
                      "Joško Gvardiol",
                      "Bryan Mbeumo",
                      "Cole Palmer",
                      "Morgan Rogers",
                      "Alex Iwobi",
                      "Mohamed Salah",
                      "Yoane Wissa",
                      "Alexander Isak",
                      "Jørgen Strand Larsen"
                      }

In [4]:
filesSorted = getDataFilesSorted()
dataFileName = filesSorted[-1]
print(dataFileName)

./data/player_stats/data_34.csv


In [5]:
with open(scores_filename, "r") as f:
    all_player_data = pd.read_json(f,orient="records")

In [6]:
print(all_player_data)

      id                          name  cost  ict_index  total_points  \
0      0         Fábio Ferreira Vieira    54        0.0             0   
1      1     Gabriel Fernando de Jesus    65       52.6            42   
2      2  Gabriel dos Santos Magalhães    62      108.2           117   
3      3                   Kai Havertz    77      142.6            96   
4      4                     Karl Hein    40        0.0             0   
..   ...                           ...   ...        ...           ...   
775  794                   Wes Okoduwa    40        0.0             0   
776  795              Emmanuel Agbadou    40       37.4            35   
777  797                  Nasser Djiga    45        0.5             4   
778  798              Marshall Munetsi    50       37.2            36   
779  799                   Mateus Mané    45        0.0             0   

     points_per_game  form status  starts_per_90 position team  combined  \
0                0.0   0.0      u           0.0

In [7]:
current_team = Team.fromNameSet(current_team_names, all_player_data)

Yoane Wissa a
Alexander Isak a
Dan Burn a
Ashley Young a
Mohamed Salah a
Mark Flekken a
Alex Iwobi a
Bryan Mbeumo a
Cole Palmer a
Matz Sels a
Joško Gvardiol a
Rayan Aït-Nouri a
Daniel Muñoz a
Morgan Rogers a
Jørgen Strand Larsen a


In [8]:
current_team_cost = current_team.getTotalCost()
current_team_cost

1002

In [9]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)
data = all_data["data"][SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame(data=data)

In [10]:
selectedTeamNames = selected_team_df["name"].values
selected_team = Team.fromNameSet(selectedTeamNames, all_player_data)

Nathan Collins a
Fabian Schär a
Joško Gvardiol a
Rayan Aït-Nouri a
Trevoh Chalobah a
Alexander Isak a
Matheus Santos Carneiro Da Cunha a
Yoane Wissa a
Jordan Pickford a
Mark Flekken a
Morgan Rogers a
Alex Iwobi a
Bryan Mbeumo a
Jarrod Bowen a
Jacob Murphy a


In [11]:
display(selected_team)

In [12]:
new_players = selected_team - current_team
print(new_players)


Total Score: 0

Goalkeepers:
- Jordan Pickford (Captain) 	Score: 0	Cost: 51	Fixture Difficulty: 0.0
Defenders:
- Nathan Collins (Vice Captain) 	Score: 0	Cost: 45	Fixture Difficulty: 0.0
- Fabian Schär	Score: 0	Cost: 56	Fixture Difficulty: 0.0
- Trevoh Chalobah	Score: 0	Cost: 44	Fixture Difficulty: 0.0
Attackers:
- Matheus Santos Carneiro Da Cunha	Score: 0	Cost: 70	Fixture Difficulty: 0.0
Midfielders:
- Jarrod Bowen	Score: 0	Cost: 76	Fixture Difficulty: 0.0
- Jacob Murphy	Score: 0	Cost: 53	Fixture Difficulty: 0.0


In [13]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -94


In [14]:
HEURISTIC = "combined"
MODE = SolverMode.CHEAPEST_FIRST

In [15]:
all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(9.270354901259548, 9.270354901259548)

In [16]:
START_SAMPLE_GAMEWEEK = config.CURRENT_GAMEWEEK-4
END_SAMPLE_GAMEWEEK = config.CURRENT_GAMEWEEK

In [17]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK)

new_players.recalculateFixtureDifficulty(matrix)
new_players.calculateScore(HEURISTIC)

selected_team.recalculateFixtureDifficulty(matrix)
selected_team.calculateScore(HEURISTIC)

current_team.recalculateFixtureDifficulty(matrix)
current_team.calculateScore(HEURISTIC)

In [18]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [19]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 78.23267034519404


In [20]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [21]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [22]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [23]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [24]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [25]:
newTeam = getNewTeam(current_team, selected_team, new_players)
newTeam = newTeam.toBenchTeam()
display(newTeam)

Best transfer:
Transfer from Cole Palmer -> Jacob Murphy:
Old player: Cole Palmer	Score: 38.29	Cost: 105	Fixture Difficulty: -2.3948416828253833
New player: Jacob Murphy	Score: 68.42	Cost: 53	Fixture Difficulty: -3.8433584310578564
- Cost change: -52
- Score change: 30.13
